# Project 2 - Medical Imaging 

## Imports

In [66]:
import os 
import re 

import numpy as np 
import pandas as pd 
from PIL import Image 
from matplotlib import pyplot as plt 
import seaborn as sns 

from skimage import morphology 
from skimage.transform import rotate 


from time import time 

## Global Variables

In [36]:
# Initial parameters for running the notebook 
PREPROCESS = True 
COMPUTE_FEATURES = True 

In [47]:
# Paths to directories used 

ROOT_DIR = "../"
DATA_DIR = ROOT_DIR + "data"
IM_DIR = DATA_DIR + "/example_image"
MASK_DIR = DATA_DIR + "/example_segmentation"
DIAGNOSIS_PATH = DATA_DIR + "/example_ground_truth.csv"


In [62]:
# Load all filenames into a dictionary
FILENAMES = {}
FILENAMES['images'] = sorted([IM_DIR + "/" + i for i in list(os.walk(IM_DIR))[0][2]])
FILENAMES['masks'] = sorted([f"{MASK_DIR}/{i}" for i in list(os.walk(MASK_DIR))[0][2]])
FILENAMES['image_num'] = len(FILENAMES['images'])

## Helper functions 

In [39]:
def make_sides_even(image): 
    '''
    Function to make the numbers of columns and rows in 
    an image even. 
    Input: An image 
    Output: An image
    '''
    # Convert image to numpy array
    image = np.array(image)
    
    # Check if the number of rows is even 
    if image.shape[0] % 2 != 0: 
        # Delete the first row
        image = np.delete(image,0,axis = 0)
    
    # Check if the number of columns is even 
    if image.shape[1] % 2 != 0: 
        # Delete the first column
        image = np.delete(image,0,axis = 1)
    
    # Convert numpy array back to image 
    image = Image.fromarray(image)
    # Return the updated image 
    return image

In [40]:
def filter_and_crop_image(image_path,mask_path): 
    image = Image.open(image_path) 
    mask = Image.open(mask_path)
    
    image_crop = image.crop(mask.getbbox())
    mask_crop = mask.crop(mask.getbbox())
    
    image_crop = make_sides_even(image_crop)
    mask_crop = make_sides_even(mask_crop)

    tmp_image = Image.new("RGB",image_crop.size, 0)
    filtered_image = Image.composite(image_crop,tmp_image,mask_crop)

    return filtered_image, mask_crop, image_crop

## Preprocessing

In [69]:
start = time()
if PREPROCESS: 
    try: 
        os.makedirs(DATA_DIR + "/filtered_images")
        os.makedirs(DATA_DIR + "/filtered_masks")
        os.makedirs(DATA_DIR + "/cropped_images")

    except FileExistsError:
        print("Directories exist")
    except IsADirectoryError: 
        print("Directories exist")
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"
    for i in range(FILENAMES['image_num']):
        mask_path = FILENAMES['masks'][i]
        image_path = FILENAMES['images'][i]
        image, mask, crop = filter_and_crop_image(image_path,mask_path)
        image_name = image_path.split("/")[-1].split(".")[-2]
        mask_name = mask_path.split("/")[-1].split(".")[-2]
        image.save(IMAGE_FILTER_DIR + "/" + image_name + ".jpg")
        mask.save(MASK_FILTER_DIR + "/" + mask_name + ".png")
        crop.save(IMAGE_CROP_DIR + "/" + image_name + ".jpg")
        del mask_path, image_path, image, mask, crop, image_name, mask_name
    
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in os.walk(IMAGE_FILTER_DIR)])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in os.walk(IMAGE_CROP_DIR)])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in os.walk(MASK_FILTER_DIR)])
else: 
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in os.walk(IMAGE_FILTER_DIR)])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in os.walk(IMAGE_CROP_DIR)])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in os.walk(MASK_FILTER_DIR)])

end = time()
print(f"The entire operation took {end - start:6.3f} seconds")

Directories exist
The entire operation took 60.447 seconds


## Processing 

## Feature Extraction Functions

In [ ]:
## Find the area and perimeter of the mask 
def get_area_perimeter(mask): 
    '''
    Function which takes in a mask for a
    given image and returns the area and 
    perimeter of the mask. 
    '''
    area = np.sum(mask)
    perimeter = area - np.sum(morphology.binary_erosion(mask,morphology.disk(3)))
    return area, perimeter 

In [ ]:
def get_compactness(mask): 
    '''
    Function which takes in a mask for a given 
    image, calls the get_area_perimeter function
    to get the area and perimeter, and returns a 
    compactness score based upon [CITATION NEEDED]. 
    '''
    area, perimeter = get_area_perimeter(mask) 
    compactness = perimeter ** 2 / (4 * np.pi * area)
    return compactness 

In [ ]:
def get_asymmetry(mask, rotation=180): 
    '''
    Takes in a mask for a given image, rotates it 
    180 times by one degree, compares the left and 
    right half and returns an average asymmetry score.
    '''
    
    axes = 0 
    length, width = mask.shape 
    pad_size = int(max((length, width))/2)
    if pad_size % 2 != 0: 
        pad_size += 1 
    mask = np.pad(mask,pad_size)
    
    diffs = []

    while axes * rotation < 180:
        temp_mask = rotate(mask, axes * rotation)
        length_lesion = np.nonzero(np.sum(temp_mask, axis = 0))[0][-1] - np.nonzero(np.sum(temp_mask, axis = 0))[0][0]
        left_mask = temp_mask[0: width, 0:np.nonzero(np.sum(temp_mask, axis = 0))[0][0] + length_lesion//2]
        right_mask = temp_mask[0: width, np.nonzero(np.sum(temp_mask, axis = 0))[0][0] + length_lesion//2:length]
        diffs.append(np.abs(np.sum(left_mask)-np.sum(right_mask)))
        
        axes += 1
    
    diff = np.mean(diffs)
    
    return diff / np.sum(mask)



In [ ]:
def get_average_luminance(image): 
    '''
    A function which takes in an image and
    returns the average luminance. 
    Input: A PIL Image 
    Output: Average luminance of the image
    '''
    grayscale = np.array(image.convert('L'))
    average_luminance = round(np.mean(grayscale[grayscale > 1]))
    return average_luminance

In [ ]:
def get_luminance_variability(image,measure="variance"): 
    grayscale = np.array(image.convert('L'))
    if measure == 'variance': 
        return round(np.var(grayscale[grayscale > 0]))
    elif measure == "standard_deviation": 
        return round(np.std(grayscale[grayscale > 0]))
    else: 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 

In [ ]:
def get_avg_color(image): 
    '''
    A function which takes in an image and returns 
    the average color of the image. 
    '''
    r, g, b = image.split()
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    average_color = (
        round(np.mean(r[r > 0])), 
        round(np.mean(g[g > 0])), 
        round(np.mean(b[b > 0]))
    )
    return average_color

In [ ]:
def get_color_variance(image,measure="variance"): 
    '''
    A function which takes in an image and 
    returns the variance of the color. 
    '''
    r,g,b = image.split()
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    if measure == "variance": 
        rgb = (
            np.var(r[r>0]),
            np.var(g[g>0]),
            np.var(b[b>0])
        )
    elif measure == "standard_deviation": 
        rgb = (
            np.std(r[r>0]),
            np.std(g[g>0]),
            np.std(b[b>0])
        )
    else: 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 
    return np.mean(rgb)